In [203]:
import pandas as pd
from bs4 import BeautifulSoup
import re

In [204]:

def clean_html_tags(data: str) -> str:
    """Remove HTML tags from textual data"""
    if not isinstance(data, str):
        return data  
        
    if not data:
        return
    
    soup = BeautifulSoup(data, "html.parser")
    return soup.get_text()

def remove_hashtags(text: str) -> str:
    """Remove hashtags from textual data"""
    if not isinstance(text, str):
        return text
        
    if not text:
        return
    
    return re.sub(r"#\w+", "", text)

def count_separator_rows(patterns: list, column: pd.Series) -> int:
    """
    Count rows in column with with specified separators. 
    Prints the total number of rows in the column, the number of rows with each separator and the total number of rows with any of the separators.
    """
    
    column_len = len(column)
    print(f"Number of rows in column: {column_len}")

    rows_with_pattern_separators = {}
    for pattern in patterns:
        rows_with_pattern_separator = column.str.contains(pattern, regex=True, na=False)
        rows_with_pattern_separators[pattern] = rows_with_pattern_separator
        print(f"Number of rows with {repr(pattern)} separator: {rows_with_pattern_separator.sum()}")
        
    # series of false with len of step_column. we use OR on it and count the number of rows with any of the separators listed
    rows_with_either = pd.Series([False] * column_len)

    for pattern, pattern_matches in rows_with_pattern_separators.items():
        rows_with_either = rows_with_either | pattern_matches
        
    count_either = rows_with_either.sum()
    print(f"Number of rows with either ',.' or newlines: {count_either}")
    
    return count_either

def split_steps(steps: str, separators: list, ends_with_dot: bool = True) -> list:
    """Split textual data into a list based on provided separators."""
    if not isinstance(steps, str):
        return steps
    
    pattern = "|".join(separators)
    steps_list = re.split(pattern, steps)
    
    # ensure consistency, all steps end with a dot
    steps_list = [step.strip() for step in steps_list if step.strip()]
    
    if ends_with_dot:
        for i in range(len(steps_list)):
            if not steps_list[i].endswith('.'):
                steps_list[i] += '.'
    
    return steps_list
    

### Dataset Loading and Initial Processing

In [205]:
df = pd.read_csv("../data/Recipes.csv")
df.sample(5)

,id,name,name.1,author_note,ingredients,steps
67,79,Hovězí vývar,Roman Vaněk,S Rohlíkem jsme dali dohromady výběr nejoblíbe...,"Bobkový list, Černý pepř celý, Česnek, Cibule,...","Jakmile se pěna přestane tvořit, přidáme zarov..."
32,43,Makarony se sýrem,Roman Vaněk,S Rohlíkem jsme dali dohromady výběr nejoblíbe...,"Česnek, Cheddar bloček, Cheddar Extra Mature, ...",Dvě třetiny bešamelu smícháme v míse s uvařený...
71,85,Bramborové noky,Roman Vaněk,S Rohlíkem jsme dali dohromady výběr nejoblíbe...,"Bramborový škrob, Brambory varný typ C, Hrubá ...",Do většího hrnce dáme vařit dobře osolenou vod...
3,14,Kuřecí polévka s kokosovým mlékem,Roman Vaněk,S Rohlíkem jsme dali dohromady výběr nejoblíbe...,"Čerstvý koriandr, Červená Chilli paprička, Čes...",Maso vyndáme z hrnce a nakrájíme na plátky sil...
28,39,Kuře s rajčaty a olivami,Roman Vaněk,S Rohlíkem jsme dali dohromady výběr nejoblíbe...,"Celé kuře, Černé olivy bez pecky, Čerstvě drce...",Do kastrolu vyskládáme zpět opečené maso kůží ...


In [206]:
df = df.rename(columns={"name.1": "author_name"}) # rename column as there were duplicate names

### Data Cleaning

During manual inspection, I found that "author_note" column had hashtags and html tags in it. Let's ensure that all columns won't have them.

In [207]:
for column in df.columns:        
    df[column] = df[column].apply(clean_html_tags)
    df[column] = df[column].apply(remove_hashtags)
    
df.sample(5)

,id,name,author_name,author_note,ingredients,steps
61,73,Těstoviny s pestem,Roman Vaněk,S Rohlíkem jsme dali dohromady výběr nejoblíbe...,"Čerstvá bazalka, Čerstvě drcený pepř, Česnek, ...","Bazalku pokrájíme., Do hrnce dáme vařit 3 l vo..."
31,42,Květáková polévka,Roman Vaněk,S Rohlíkem jsme dali dohromady výběr nejoblíbe...,"Bílý pepř, Cibule, Hladká mouka, Květák, Máslo...","1,1 l vody přivedeme v hrnci k varu a osolíme ..."
65,77,Asijský burger,Roman Vaněk,S Rohlíkem jsme dali dohromady výběr nejoblíbe...,"Arašídové máslo, Bageta, Čerstvě drcený pepř, ...",Bagety podélně rozřízneme a na řezné ploše je ...
4,15,Kuřecí vývar,Roman Vaněk,S Rohlíkem jsme dali dohromady výběr nejoblíbe...,"Bobkový list, Černý pepř celý, Čerstvý tymián,...",Hotový vývar přecedíme přes jemné síto a přípa...
77,91,Velikonoční beránek,Roman Vaněk,S Rohlíkem jsme dali dohromady výběr nejoblíbe...,"Cukr krupice, Hrubá mouka, Káva vhodná k tomut...",Hotového beránka necháme zchladnout ve formě a...


During the manual inspection of the dataset, it was found that the steps of cooking are separated either with ",." or new lines. 
New lines usually are respresented with:

- \n\n and \n
-  or \r\n 

Based on compilation environment. While single new lines appear to function as "sub steps" separators in some cases, it is still better to divide on them, to make steps more granular. 

Let us find out whether it is applicable to every single recipe, so we won't miss recipes with other types of separators: 

In [208]:
df["steps"] = df["steps"].str.replace('\r\n', '\n') # replace windows newline with unix newline
df["steps"] = df["steps"].str.replace(r'\n\s*\n+', '\n\n', regex=True)
print(str(count_separator_rows([".,", "\n\n", "\n"], df["steps"])))

Number of rows in column: 100
Number of rows with '.,' separator: 100
Number of rows with '\n\n' separator: 54
Number of rows with '\n' separator: 59
Number of rows with either ',.' or newlines: 100
100


Let's separate steps into list: 

In [209]:
df["steps"] = df["steps"].apply(split_steps, args=([r"\.,", r"\n\n", r"\n"],))
df.head()

,id,name,author_name,author_note,ingredients,steps
0,10,Hovězí guláš s karlovarským knedlíkem,Roman Vaněk,S Rohlíkem jsme dali dohromady výběr nejoblíbe...,"Bobkový list, Drcený kmín, Hovězí plec vcelku,...",[Cibuli nakrájíme nahrubo. Maso nakrájíme na k...
1,11,Bábovka s ořechy,Roman Vaněk,S Rohlíkem jsme dali dohromady výběr nejoblíbe...,"Cukr krupice, Hrubá mouka, Kakao, Káva vhodná ...","[Sníh z bílků musí být tak tuhý, že při obráce..."
2,12,Hrášková krémová polévka,Roman Vaněk,S Rohlíkem jsme dali dohromady výběr nejoblíbe...,"Cibule, Cukr krupice, Kuřecí vývar, Máslo, Mát...","[Pokud chcete, aby polévka získala svěží zelen..."
3,14,Kuřecí polévka s kokosovým mlékem,Roman Vaněk,S Rohlíkem jsme dali dohromady výběr nejoblíbe...,"Čerstvý koriandr, Červená Chilli paprička, Čes...",[Maso vyndáme z hrnce a nakrájíme na plátky si...
4,15,Kuřecí vývar,Roman Vaněk,S Rohlíkem jsme dali dohromady výběr nejoblíbe...,"Bobkový list, Černý pepř celý, Čerstvý tymián,...",[Hotový vývar přecedíme přes jemné síto a příp...


### Handling Different Dataset Types

In the provided dataset, the "author_note" column contains promotional, repetitive content. While future iterations of the dataset might include recipe-related information, this is not the case for now. Therefore, dropping this column is justified. 

This action improves the efficiency of future usage of the dataset for RAG, by reducing context size for LLM and reducing "noise" in embedding vectors for Vector Search.

In [210]:
df.drop(columns=["author_note"], inplace=True)

However, there arises the problem. The plan is to create retrieval mechanism that will allow the chatbot not only to "retell" the recipe, but also retrieve the full recipe from the database and show it to the user in a formatted way, to be as precise as possible and minimise hallucinations. In this case, promotional messages would be benefitial. The solution for this 

As we currently have no metadata for the dataset, there is no way to determine the purpose of the "id" field. 

### Save Datasets

In [ ]:
df_for_rag.to_csv("../data/processed/Recipes_processed_rag.csv", index=False)
df_for_retrieval.to_csv("../data/processed/Recipes_processed_retrieval.csv", index=False)